In [1]:
import numpy as np
import pandas as pd

from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import median_absolute_error

In [2]:
#properties = pd.read_csv('trainPreProcess.csv')
#testing = pd.read_csv('testPreProcess.csv')
properties = pd.read_csv('martinTrain.csv')
testing = pd.read_csv('martinTest.csv')

In [3]:
corr = properties.corr()
corr.sort_values(["price_aprox_usd"], ascending = False, inplace = True)
print(corr.price_aprox_usd)

price_aprox_usd          1.000000
pileta                   0.204898
playroom                 0.116692
lavadero                 0.113153
cancha                   0.093863
gimnasio                 0.091884
jardin                   0.084994
madero                   0.073849
seguridad                0.066096
garage                   0.063638
excelente                0.062979
grande                   0.062464
shopping                 0.058566
living                   0.056066
limpieza                 0.044945
baja                     0.041951
barrio cerrado           0.037987
parque                   0.034563
terraza                  0.028343
surface_covered_in_m2    0.013598
patio                    0.009020
parrila                  0.004129
cocina                  -0.002722
autopista               -0.008687
place_name_encoded      -0.015104
Unnamed: 0.1            -0.018176
Unnamed: 0              -0.018758
 frente                 -0.026820
tren                    -0.031106
luminoso      

In [4]:
#tomo pocos datos porque se hace muy lento el algoritmo si no (por ahora)
train = properties.sample(2000)
test = testing

In [5]:
#quito unnamed:0 y created_on, y dejo price al final, id al principio
#train = train.loc[:, ['id', 'property_type_encoded', 'place_name_encoded', 'state_name_encoded', 'lat', 'lon', 'surface_covered_in_m2', 'surface_total_in_m2', 'floor', 'rooms', 'distancia_obelisco', 'distancia_minima_subte', 'distancia_minima_privado', 'distancia_minima_publico', 'price']]

#OPTIMIZACION: quito distancias, lon, state_name_encoded y property_type_encoded
train = train.loc[:, ['id', 'pileta', 'playroom', 'lavadero', 'cancha', 'gimnasio', 'jardin', 'madero', 'seguridad', 'garage', 'excelente', 'grande', 'shopping', 'living', 'limpieza', 'baja', 'barrio cerrado', 'parque', 'terraza', 'surface_covered_in_m2', 'patio', 'parrila', 'cocina', 'autopista', 'place_name_encoded', 'tren', 'luminoso', 'centro', 'nuevo', 'contrafrente', 'cuadras', 'monoambiente', 'price_aprox_usd']]
test = test.loc[:, ['id', 'pileta', 'playroom', 'lavadero', 'cancha', 'gimnasio', 'jardin', 'madero', 'seguridad', 'garage', 'excelente', 'grande', 'shopping', 'living', 'limpieza', 'baja', 'barrio cerrado', 'parque', 'terraza', 'surface_covered_in_m2', 'patio', 'parrila', 'cocina', 'autopista', 'place_name_encoded', 'tren', 'luminoso', 'centro', 'nuevo', 'contrafrente', 'cuadras', 'monoambiente']]
longitud = len(test.columns)
print(longitud)

32


In [6]:
#XGBoost funciona con matrices, no con Df
train = train.as_matrix()
test = test.as_matrix()

In [7]:
#separo price
X_train = train[:,0:longitud]
Y_train = train[:,longitud]
X_test = test[:,0:longitud]
#Y_test = test[:,longitud]

In [8]:
#tomo los ids del test aparte para despues
test_ids = X_test[:,0]

In [9]:
#quito los ids de los sets, XGBoost solo trabaja con floats e ints
X_test = X_test[:,1:longitud]
X_train = X_train[:,1:longitud]

In [10]:
#armo el modelo (va a tomarse su tiempo con mas de 1000)
model = XGBClassifier(n_estimators=150, max_depth=4)
model.fit(X_train, Y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='multi:softprob', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [11]:
predictions = model.predict(X_test)

In [12]:
#score = mean_squared_error(Y_test, predictions)
#score

In [13]:
#median_absolute_error(Y_test, predictions)

In [14]:
#armo un csv con id, precio real, y precio predicho por XGBoost
predictionsflat = predictions.flatten()
idsflat = test_ids.flatten()
submissionDf = pd.DataFrame()
#submissionDf['id'] = pd.Series(test_ids)
submissionDf['id'] = pd.Series(test_ids.astype('int32'))
submissionDf['price_usd'] = pd.Series(predictionsflat)
#submissionDf['precio_xgb'] = pd.Series(predictionsflat)
submissionDf.set_index(['id'], inplace=True)
submissionDf.to_csv('predicciones_XGB.csv')

In [15]:
submissionDf.head(10)

,price_usd
id,
3632,170000
3633,116000
2263404,140000
2263405,80000
2263406,80000
2263407,80000
2263408,80000
2263409,80000
2263410,125000


In [16]:
#testnuevo= pd.merge(testing, submissionDf, on='id', how='outer')
#testnuevo.to_csv('lexicoXGB.csv')